# 🎯 Dashboard Analisi Complementarità Calciatori
# 
# Questo notebook analizza la complementarità tra calciatori basandosi su diversi fattori.

In [14]:
# Prima cella: Importazioni
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [26]:
# Seconda cella: Funzioni di utilità
def calculate_age(birth_date_str):
    """Calcola l'età da una stringa di data di nascita"""
    if pd.isna(birth_date_str):
        return 0
    try:
        birth_date = datetime.strptime(birth_date_str.split(' ')[0], '%Y-%m-%d')
        today = datetime.now()
        age = today.year - birth_date.year
        if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
            age -= 1
        return age
    except:
        return 0

def format_currency(value):
    """Formatta i valori monetari"""
    if pd.isna(value) or value == 0:
        return "N/A"
    if value >= 1000000:
        return f"€{value/1000000:.1f}M"
    elif value >= 1000:
        return f"€{value/1000:.1f}K"
    return f"€{value:.0f}"

def calculate_bmi(weight_kg, height_m):
    """Calcola il BMI"""
    if height_m > 0 and weight_kg > 0:
        return weight_kg / (height_m ** 2)
    return 0

In [16]:
# Terza cella: Caricamento dati
# Caricamento dati
players_df = pd.read_csv('../Datasets/SNA/players.csv')
clubs_df = pd.read_csv('../Datasets/SNA/clubs.csv')
transfers_df = pd.read_csv('../Datasets/SNA/transfers.csv')

# Pulizia e preparazione
players_df['age'] = players_df['date_of_birth'].apply(calculate_age)

# Filtro giocatori attivi (ultimi 2 anni)
active_players = players_df[players_df['last_season'] >= 2022].copy()

# Aggiungi informazioni sul club
active_players = active_players.merge(
    clubs_df[['club_id', 'name', 'domestic_competition_id']],
    left_on='current_club_id',
    right_on='club_id',
    how='left'
)

# Mostra le prime righe dei dati
print("Numero di giocatori attivi:", len(active_players))
active_players.head()

Numero di giocatori attivi: 12946


,player_id,first_name,last_name,name_x,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,age,club_id,name_y,domestic_competition_id
0,2857,Eldin,Jakupovic,Eldin Jakupovic,2022,29,eldin-jakupovic,Jugoslawien (SFR),Kozarac,Switzerland,...,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/eldin-jakupovi...,GB1,Everton Football Club,100000.0,1500000.0,40,29,Everton Football Club,GB1
1,3333,James,Milner,James Milner,2024,1237,james-milner,England,Leeds,England,...,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/james-milner/p...,GB1,Brighton and Hove Albion Football Club,1000000.0,21000000.0,38,1237,Brighton and Hove Albion Football Club,GB1
2,3455,Zlatan,Ibrahimović,Zlatan Ibrahimović,2022,5,zlatan-ibrahimović,Sweden,Malmö,Sweden,...,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/zlatan-ibrahim...,IT1,Associazione Calcio Milan,2000000.0,46000000.0,43,5,Associazione Calcio Milan,IT1
3,4311,Maarten,Stekelenburg,Maarten Stekelenburg,2022,610,maarten-stekelenburg,NaN,Haarlem,NaN,...,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/maarten-stekel...,NL1,AFC Ajax Amsterdam,75000.0,11000000.0,42,610,AFC Ajax Amsterdam,NL1
4,4391,Boy,Waterman,Boy Waterman,2023,383,boy-waterman,Netherlands,Lelystad,Netherlands,...,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/boy-waterman/p...,NL1,Eindhovense Voetbalvereniging Philips Sport Ve...,50000.0,3000000.0,40,383,Eindhovense Voetbalvereniging Philips Sport Ve...,NL1


In [17]:
# Funzione di compatibilità migliorata
def calculate_player_compatibility(player1, player2):
    """
    Calcola il punteggio di compatibilità tra due giocatori con criteri più stringenti
    """
    score = 0
    reasons = []
    
    # 1. Complementarità Posizionale (0-35 punti)
    position_complementarity = {
        'Centre-Forward': ['Attacking Midfield', 'Central Midfield', 'Left Winger', 'Right Winger'],
        'Attacking Midfield': ['Centre-Forward', 'Central Midfield', 'Defensive Midfield'],
        'Central Midfield': ['Centre-Forward', 'Defensive Midfield', 'Centre-Back'],
        'Defensive Midfield': ['Centre-Back', 'Central Midfield', 'Attacking Midfield'],
        'Left Winger': ['Centre-Forward', 'Left-Back', 'Central Midfield'],
        'Right Winger': ['Centre-Forward', 'Right-Back', 'Central Midfield'],
        'Left-Back': ['Left Winger', 'Centre-Back', 'Defensive Midfield'],
        'Right-Back': ['Right Winger', 'Centre-Back', 'Defensive Midfield'],
        'Centre-Back': ['Defensive Midfield', 'Right-Back', 'Left-Back']
    }
    
    if player1['sub_position'] in position_complementarity and player2['sub_position'] in position_complementarity[player1['sub_position']]:
        score += 35
        reasons.append(f"Posizioni altamente complementari: {player1['sub_position']} - {player2['sub_position']}")
    
    # 2. Piede preferito (0-15 punti)
    if player1['foot'] and player2['foot'] and player1['foot'] != player2['foot']:
        score += 15
        reasons.append(f"Complementarità piedi: {player1['foot']} - {player2['foot']}")
    
    # 3. Età e Esperienza (0-15 punti)
    age_diff = abs(player1['age'] - player2['age'])
    if 2 <= age_diff <= 4:
        score += 15
        reasons.append(f"Mix ideale esperienza-gioventù: {age_diff} anni di differenza")
    
    # 4. Altezza complementare (0-10 punti)
    if player1['height_in_cm'] and player2['height_in_cm']:
        height_diff = abs(player1['height_in_cm'] - player2['height_in_cm'])
        if 5 <= height_diff <= 15:
            score += 10
            reasons.append("Complementarità fisica")
    
    # 5. Valore di mercato bilanciato (0-15 punti)
    if player1['market_value_in_eur'] > 0 and player2['market_value_in_eur'] > 0:
        market_value_ratio = min(player1['market_value_in_eur'], player2['market_value_in_eur']) / max(player1['market_value_in_eur'], player2['market_value_in_eur'])
        if market_value_ratio > 0.3:
            score += 15
            reasons.append("Valori di mercato ben bilanciati")
    
    # 6. Nazionalità diverse ma stesso campionato (0-10 punti)
    if (player1['country_of_citizenship'] != player2['country_of_citizenship'] and 
        player1['current_club_domestic_competition_id'] == player2['current_club_domestic_competition_id']):
        score += 10
        reasons.append("Diversità culturale nello stesso campionato")
    
    return score, reasons

In [29]:
# Quinta cella: Analisi di un giocatore specifico
# Mostra alcuni giocatori disponibili nel dataset
print("Alcuni giocatori disponibili nel dataset:")
sample_players = active_players.apply(lambda x: f"{x['first_name']} {x['last_name']}", axis=1).sample(10)
print(sample_players.tolist())

# Seleziona un giocatore di esempio
player_full_name = "Matteo Darmian" 
player = active_players[
    (active_players['first_name'] + " " + active_players['last_name']) == player_full_name
].iloc[0]

# Trova i giocatori compatibili
compatibilities = []
for _, potential_match in active_players.iterrows():
    potential_match_name = f"{potential_match['first_name']} {potential_match['last_name']}"
    if potential_match_name != player_full_name:
        score, reasons = calculate_player_compatibility(player, potential_match)
        compatibilities.append({
            'name': potential_match_name,
            'score': score,
            'reasons': reasons,
            'position': potential_match['position'],
            'market_value': potential_match['market_value_in_eur'],
            'age': potential_match['age']
        })

# Ordina per punteggio
compatibilities.sort(key=lambda x: x['score'], reverse=True)

# Mostra i top 5 giocatori più compatibili
print(f"\nTop 5 giocatori più compatibili con {player_full_name}:")
for i, comp in enumerate(compatibilities[:5]):
    print(f"\n{i+1}. {comp['name']} (Score: {comp['score']}/100)")
    print(f"   Posizione: {comp['position']}")
    print(f"   Età: {comp['age']}")
    print(f"   Valore di mercato: {format_currency(comp['market_value'])}")
    print("   Ragioni della compatibilità:")
    for reason in comp['reasons']:
        print(f"   - {reason}")

Alcuni giocatori disponibili nel dataset:
['James Abankwah', 'Ondrej Duda', 'Brent Vugts', 'Kevin Kampl', 'Bas Dost', 'Can Uzun', 'Colin Dagba', 'Abde Ezzalzouli', 'Andres Sfait', 'Cyle Larin']

Top 5 giocatori più compatibili con Matteo Darmian:

1. Pablo Marí (Score: 100/100)
   Posizione: Defender
   Età: 31
   Valore di mercato: €3.5M
   Ragioni della compatibilità:
   - Posizioni altamente complementari: Right-Back - Centre-Back
   - Complementarità piedi: right - left
   - Mix ideale esperienza-gioventù: 4 anni di differenza
   - Complementarità fisica
   - Valori di mercato ben bilanciati
   - Diversità culturale nello stesso campionato

2. Junior Messias (Score: 100/100)
   Posizione: Attack
   Età: 33
   Valore di mercato: €2.0M
   Ragioni della compatibilità:
   - Posizioni altamente complementari: Right-Back - Right Winger
   - Complementarità piedi: right - left
   - Mix ideale esperienza-gioventù: 2 anni di differenza
   - Complementarità fisica
   - Valori di mercato ben 

In [32]:
def create_advanced_visualization(player, compatibilities):
    """
    Crea visualizzazione radar per l'analisi di complementarità tra calciatori
    con punteggi standardizzati (0-20) per ogni categoria
    """
    fig1 = go.Figure()
    
    colors = [
        'rgba(255, 107, 107, 0.8)',  # rosso
        'rgba(78, 205, 196, 0.8)',   # verde acqua
        'rgba(69, 183, 209, 0.8)',   # azzurro
        'rgba(150, 206, 180, 0.8)',  # verde chiaro
        'rgba(255, 238, 173, 0.8)'   # giallo chiaro
    ]
    
    categories = [
        'Compatibilità Posizionale',
        'Complementarità Piede',
        'Mix Esperienza',
        'Compatibilità Fisica',
        'Equilibrio Economico'
    ]
    
    for idx, comp in enumerate(compatibilities[:5]):
        # Calcolo punteggi standardizzati (0-20)
        pos_score = 20 if 'Posizioni altamente complementari' in ' '.join(comp['reasons']) else (
            10 if 'Posizioni complementari' in ' '.join(comp['reasons']) else 0
        )
        
        foot_score = 20 if 'Complementarità piedi' in ' '.join(comp['reasons']) else 0
        
        age_diff = abs(player['age'] - comp['age'])
        exp_score = 20 if 2 <= age_diff <= 4 else (
            10 if 1 <= age_diff <= 5 else 0
        )
        
        # Nuovo calcolo compatibilità fisica
        height_diff = abs(player['height_in_cm'] - comp.get('height_in_cm', 0))
        weight_diff = abs(player.get('weight_kg', 0) - comp.get('weight_kg', 0))
        
        # Calcolo BMI
        player_bmi = calculate_bmi(player.get('weight_kg', 0), player['height_in_cm']/100)
        comp_bmi = calculate_bmi(comp.get('weight_kg', 0), comp.get('height_in_cm', 0)/100)
        bmi_diff = abs(player_bmi - comp_bmi)
        
        # Punteggio fisico basato su più parametri
        phys_score = 0
        if 5 <= height_diff <= 15:
            phys_score += 10
        if 5 <= weight_diff <= 15:
            phys_score += 5
        if 0 <= bmi_diff <= 2:
            phys_score += 5
        
        if player['market_value_in_eur'] > 0 and comp['market_value'] > 0:
            market_ratio = min(player['market_value_in_eur'], comp['market_value']) / max(player['market_value_in_eur'], comp['market_value'])
            eco_score = 20 if market_ratio > 0.7 else (
                15 if market_ratio > 0.4 else (
                    10 if market_ratio > 0.2 else 0
                )
            )
        else:
            eco_score = 0
            market_ratio = 0
        
        values = [pos_score, foot_score, exp_score, phys_score, eco_score]
        
        # Testi hover dettagliati
        hover_text = [
            f"<b>Compatibilità Posizionale:</b> {pos_score}/20<br>" +
            f"({comp['position']} - {player['position']})",
            
            f"<b>Complementarità Piede:</b> {foot_score}/20<br>" +
            f"({comp.get('foot', 'N/A')} - {player.get('foot', 'N/A')})",
            
            f"<b>Mix Esperienza:</b> {exp_score}/20<br>" +
            f"Differenza età: {age_diff} anni",
            
            f"<b>Compatibilità Fisica:</b> {phys_score}/20<br>" +
            f"Differenza altezza: {height_diff}cm<br>" +
            f"Differenza peso: {weight_diff}kg<br>" +
            f"Differenza BMI: {bmi_diff:.1f}",
            
            f"<b>Equilibrio Economico:</b> {eco_score}/20<br>" +
            f"Ratio valori: {market_ratio:.2f}"
            
        ]
        
        fig1.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            name=f"{comp['name']}",
            fill='toself',
            line=dict(color=colors[idx]),
            fillcolor=colors[idx],
            hovertext=hover_text,
            hoverinfo='text'
        ))
    
    # Layout ottimizzato
    fig1.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 20],
                ticktext=['0', '5', '10', '15', '20'],
                tickvals=[0, 5, 10, 15, 20],
                ticksuffix=' pts',
                title='Punteggio Base'
            ),
            angularaxis=dict(
                ticktext=categories,
                tickvals=categories,
                tickmode='array',
                direction='clockwise',
                tickangle=0
            )
        ),
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=-0.1,
            xanchor="center",
            x=0.5,
            orientation="h"
        ),
        title={
            'text': f'Analisi Complementarità con {player["first_name"]} {player["last_name"]}',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        margin=dict(t=100, b=150, r=200, l=50),
        height=800,
        width=1200,
        annotations=[
            dict(
                text='<b>Sistema di Valutazione (0-20 pts)</b><br><br>' +
                     '<b>Compatibilità Posizionale:</b><br>' +
                     '• 20 pts: Alta complementarità<br>' +
                     '• 10 pts: Media complementarità<br>' +
                     '• 0 pts: Bassa complementarità<br><br>' +
                     '<b>Complementarità Piede:</b><br>' +
                     '• 20 pts: Piedi opposti<br>' +
                     '• 0 pts: Stesso piede<br><br>' +
                     '<b>Mix Esperienza:</b><br>' +
                     '• 20 pts: 2-4 anni differenza<br>' +
                     '• 10 pts: 1-5 anni differenza<br>' +
                     '• 0 pts: Altra differenza' + 
                     '<br><br>' +
                     '<b>Compatibilità Fisica:</b><br>' +
                     '• 20 pts: 5-15 cm differenza<br>' +
                     '• 10 pts: 0-20 cm differenza<br>' +
                     '• 0 pts: Altra differenza<br><br>' +
                     '<b>Equilibrio Economico:</b><br>' +
                     '• 20 pts: Ratio > 0.7<br>' +
                     '• 15 pts: Ratio > 0.4<br>' +
                     '• 10 pts: Ratio > 0.2<br>' +
                     '• 0 pts: Ratio ≤ 0.2<br><br>' +
                     '<i>Hover sui punti per dettagli specifici</i>',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=1.2,
                y=0.95,
                bordercolor='black',
                borderwidth=1,
                bgcolor='white',
                font=dict(size=11)
            
            )
        ]
    )
    
    # Aggiungiamo l'istogramma dei top 15
    fig2 = go.Figure()
    
    # Ordiniamo i giocatori per punteggio
    top_15 = sorted(compatibilities[:15], key=lambda x: x['score'], reverse=True)
    
    # Creiamo una scala di colori basata sui punteggi
    max_score = max(x['score'] for x in top_15)
    min_score = min(x['score'] for x in top_15)
    
    # Calcoliamo i colori in base al punteggio
    colors = [
        f'rgba(255, {int(255 * (1 - (x["score"] - min_score)/(max_score - min_score)))}, 0, 0.7)'
        for x in top_15
    ]
    
    fig2.add_trace(go.Bar(
        x=[f"{x['name']}<br>({x['position']})" for x in top_15],
        y=[x['score'] for x in top_15],
        marker_color=colors,
        text=[f"{x['score']:.1f}" for x in top_15],
        textposition='auto',
        hovertemplate=(
            "<b>%{x}</b><br>" +
            "Punteggio: %{y:.1f}<br>" +
            "Età: %{customdata[0]} anni<br>" +
            "Valore: %{customdata[1]}<br>" +
            "<extra></extra>"
        ),
        customdata=[[x['age'], format_currency(x['market_value'])] for x in top_15]
    ))
    
    fig2.update_layout(
        title={
            'text': f'Top 15 Giocatori Compatibili con {player["first_name"]} {player["last_name"]}',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title="Giocatore",
        yaxis_title="Punteggio di Compatibilità",
        yaxis_range=[0, 100],
        height=500,
        width=1200,
        margin=dict(t=100, b=150, r=50, l=50),
        showlegend=False,
        xaxis_tickangle=-45
    )
    
    return fig1, fig2

# Aggiungiamo una funzione per mostrare un sommario testuale
def print_compatibility_summary(player, compatibilities):
    """
    Stampa un sommario dettagliato delle compatibilità
    """
    print(f"\nANALISI DETTAGLIATA COMPATIBILITÀ PER {player['first_name']} {player['last_name']}")
    print(f"Posizione: {player['position']} ({player['sub_position']})")
    print(f"Età: {player['age']} anni")
    print(f"Valore di mercato: {format_currency(player['market_value_in_eur'])}")
    print("\nTOP 5 GIOCATORI COMPATIBILI:")
    
    for i, comp in enumerate(compatibilities[:5], 1):
        print(f"\n{i}. {comp['name']} - Score: {comp['score']}/100")
        print(f"   {'='*50}")
        print(f"   Posizione: {comp['position']}")
        print(f"   Età: {comp['age']} anni")
        print(f"   Valore di mercato: {format_currency(comp['market_value'])}")
        print("   Fattori di compatibilità:")
        for reason in comp['reasons']:
            print(f"   • {reason}")

# Utilizzo
fig1, fig2 = create_advanced_visualization(player, compatibilities)
print_compatibility_summary(player, compatibilities)
fig1.show()
fig2.show()


ANALISI DETTAGLIATA COMPATIBILITÀ PER Matteo Darmian
Posizione: Defender (Right-Back)
Età: 35 anni
Valore di mercato: €4.0M

TOP 5 GIOCATORI COMPATIBILI:

1. Pablo Marí - Score: 100/100
   Posizione: Defender
   Età: 31 anni
   Valore di mercato: €3.5M
   Fattori di compatibilità:
   • Posizioni altamente complementari: Right-Back - Centre-Back
   • Complementarità piedi: right - left
   • Mix ideale esperienza-gioventù: 4 anni di differenza
   • Complementarità fisica
   • Valori di mercato ben bilanciati
   • Diversità culturale nello stesso campionato

2. Junior Messias - Score: 100/100
   Posizione: Attack
   Età: 33 anni
   Valore di mercato: €2.0M
   Fattori di compatibilità:
   • Posizioni altamente complementari: Right-Back - Right Winger
   • Complementarità piedi: right - left
   • Mix ideale esperienza-gioventù: 2 anni di differenza
   • Complementarità fisica
   • Valori di mercato ben bilanciati
   • Diversità culturale nello stesso campionato

3. Pablo Sarabia - Score: 9